In [13]:
import json, pymongo, os, re

In [14]:
class Connection:
    """Class that connects to the database creating all necessary methods for connection and closing connection, as well as the base database for adding new HTS records and the string_dict collection too
    """

    def __init__(self, db_path: str):

        self.client = pymongo.MongoClient(db_path)
        self.db = self.client['hts']
        self.collection_records = self.db['hts_records']
        self.collection_string_dict = self.db['string_dict']
        

    def closeConnection(self):
        self.client.close()

def addHTSRecord(folder_path: str, record: str, collection: pymongo.collection.Collection):

    test = False

    with open(f'{folder_path}{record}') as sec:


        data = json.load(sec)
        header = re.sub(r'\.json', '', record)

        hts_record = {
            'header': header,
            'data': data
        }

        try:
            collection.insert_one(hts_record)
        except:
            print(f'Error creating record in db for file {sec}')

def addStrRecord(str_dict: list, key: str, collection: pymongo.collection.Collection):

    try:
        record = {
            'string': key,
            'chaps': str_dict
        }
        collection.insert_one(record)
    except:
        print(f'Error creating record in db for str_dict key: {key}')


In [15]:
def createDatabase(path_records: str, path_string_dict: str, file_list: list, connection: Connection):

    #Create Collection of records
    for file in file_list:
        addHTSRecord(path_records, file, connection.collection_records)
    
    #Create Collection of string_dict
    with open(f'{path_string_dict}string_dict.json', 'r') as string_file:

        string_dict_raw = json.load(string_file)

        for key in string_dict_raw.keys():
            addStrRecord(string_dict_raw[key], key, connection.collection_string_dict)

    connection.closeConnection()

In [16]:
folder_path = '../db_hts/temp/NEW_test_files/'
string_folder_path = '../db_hts/temp/NEW_test_string_dict/'

file_list = os.listdir(folder_path)

connection = Connection('mongodb://127.0.0.1:27017')

createDatabase(folder_path, string_folder_path, file_list, connection)